# Explore Ephys Sessions, Experiments and Conditions in the DataJoint Workflow


This notebook aims to explore various queries related to organoid IDs, experiment times, and drug applications using the pipeline. New cells can be added to examine each table's dependencies and attributes.

**_Note:_**

- The examples in this notebook use a sample dataset. Replace these entries with your actual database entries to access and analyze your data.



### **Key Steps**

- **Setup**

- **Step 1: Querying Ephys Sessions as Batches per Conditions**

    - **1.1. Selection of Keys for One Organoid Under Different Conditions and for a Specific Duration**

    - **1.2: Query for All Organoids Under Different Conditions**

    - **1.3: Queries for Batches Under Different Conditions**

- **Step 2: Fetching the corresponding Intan Files for an Ephys Session**


#### **Setup**


First, import the necessary packages for the data pipeline and essential schemas.


In [1]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

In [2]:
import datajoint as dj
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [3]:
from workflow.pipeline import culture, ephys

[2024-07-23 01:10:42,357][INFO]: Connecting milagros@db.datajoint.com:3306
[2024-07-23 01:10:44,611][INFO]: Connected milagros@db.datajoint.com:3306


#### **Step 1: Querying Ephys Sessions as Batches per Conditions**

##### **1.1: Select Keys for One Organoid Under Different Conditions and for a Specific Duration**


The `Experiment` table contains manually inserted data on experiments conducted on each organoid under various drug conditions. This table serves as a reference of experiment start and end times for each condition.

Let's have a look at the different conditions and experiment times for organoid `O09`:

In [4]:
exp_query = (culture.Experiment & 'organoid_id="O09"').proj(
    "experiment_end_time",
    "drug_name",
    "experiment_plan",
)
exp_query

organoid_id e.g. O17,experiment_start_time,experiment_end_time,drug_name,"experiment_plan e.g. mrna lysate, oct, protein lysate, or matrigel embedding, ephys, tracing"
O09,2023-05-03 17:33:00,2023-05-18 12:15:00,Control,ephys
O09,2023-05-18 12:25:00,2023-05-18 18:15:00,4-AP,ephys
O09,2023-05-18 18:15:00,2023-05-19 09:30:00,No Drug,ephys
O09,2023-05-19 09:30:00,2023-05-19 15:35:00,Bicuculline,ephys
O09,2023-05-19 15:45:00,2023-05-20 15:40:00,Tetrodotoxin,ephys


The specific ephys sessions are defined in the `EphysSession` table as `session_type="spike_sorting"`.

In [5]:
ephys.EphysSession & "organoid_id='O09'" & "session_type='spike_sorting'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,session_type
O09,2023-05-03 17:33:00,0,2023-05-03 17:38:00,2023-05-03 17:53:00,spike_sorting
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:26:00,2023-05-18 12:31:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:27:00,2023-05-18 12:32:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:34:00,2023-05-18 12:39:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:45:00,2023-05-18 12:50:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:52:00,2023-05-18 12:57:00,spike_sorting
O09,2023-05-18 12:25:00,0,2023-05-18 12:54:00,2023-05-18 12:55:00,spike_sorting


Also, we want to select from a set of sessions that have the same ephys duration, in this case, 15min:

In [6]:
ephys_sessions_query = (
    exp_query * ephys.EphysSession & "session_type='spike_sorting'"
).proj(
    experiment_end_time="experiment_end_time",
    drug_name="drug_name",
    duration="TIMEDIFF(end_time, start_time)",
)
ephys_sessions_query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-03 17:33:00,0,2023-05-03 17:38:00,2023-05-03 17:53:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:25:00,2023-05-18 12:30:00,2023-05-18 18:15:00,4-AP,0:05:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:26:00,2023-05-18 12:31:00,2023-05-18 18:15:00,4-AP,0:05:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:27:00,2023-05-18 12:32:00,2023-05-18 18:15:00,4-AP,0:05:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:34:00,2023-05-18 12:39:00,2023-05-18 18:15:00,4-AP,0:05:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:45:00,2023-05-18 12:50:00,2023-05-18 18:15:00,4-AP,0:05:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:52:00,2023-05-18 12:57:00,2023-05-18 18:15:00,4-AP,0:05:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:54:00,2023-05-18 12:55:00,2023-05-18 18:15:00,4-AP,0:01:00


In [7]:
ephys_sessions_15min_query = ephys_sessions_query & 'duration LIKE "%15%"'
ephys_sessions_15min_query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-03 17:33:00,0,2023-05-03 17:38:00,2023-05-03 17:53:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 17:55:00,2023-05-18 18:10:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00
O09,2023-05-19 09:30:00,0,2023-05-19 09:35:00,2023-05-19 09:50:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O09,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O09,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00
O09,2023-05-19 15:45:00,0,2023-05-20 15:20:00,2023-05-20 15:35:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00


Now, it is straightforward to choose a key for each of the drug conditions:

##### Select One Session for Control (key1)


In [8]:
ephys_sessions_15min_query & "drug_name='Control'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-03 17:33:00,0,2023-05-03 17:38:00,2023-05-03 17:53:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00


In [9]:
key1 = (
    ephys_sessions_15min_query
    & "drug_name='Control'"
    & "start_time='2023-05-18 11:55:00'"
).fetch1("KEY")
key1

{'organoid_id': 'O09',
 'experiment_start_time': datetime.datetime(2023, 5, 3, 17, 33),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 18, 11, 55),
 'end_time': datetime.datetime(2023, 5, 18, 12, 10)}

##### Select One Session for 4-AP (key2)


In [10]:
ephys_sessions_15min_query & "drug_name='4-AP'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 17:55:00,2023-05-18 18:10:00,2023-05-18 18:15:00,4-AP,0:15:00


In [11]:
key2 = (
    ephys_sessions_15min_query & "drug_name='4-AP'" & "start_time='2023-05-18 12:30:00'"
).fetch1("KEY")
key2

{'organoid_id': 'O09',
 'experiment_start_time': datetime.datetime(2023, 5, 18, 12, 25),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 18, 12, 30),
 'end_time': datetime.datetime(2023, 5, 18, 12, 45)}

##### Select One Session for No Drug (key3)


In [12]:
ephys_sessions_15min_query & "drug_name='No Drug'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00


In [13]:
key3 = (ephys_sessions_15min_query & "drug_name='No Drug'").fetch1("KEY")
key3

{'organoid_id': 'O09',
 'experiment_start_time': datetime.datetime(2023, 5, 18, 18, 15),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 19, 9, 10),
 'end_time': datetime.datetime(2023, 5, 19, 9, 25)}

##### Select One Session for Bicuculline (key4)


In [14]:
ephys_sessions_15min_query & "drug_name='Bicuculline'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-19 09:30:00,0,2023-05-19 09:35:00,2023-05-19 09:50:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O09,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00


In [15]:
key4 = (
    ephys_sessions_15min_query
    & "drug_name='Bicuculline'"
    & "start_time='2023-05-19 15:15:00'"
).fetch1("KEY")
key4

{'organoid_id': 'O09',
 'experiment_start_time': datetime.datetime(2023, 5, 19, 9, 30),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 19, 15, 15),
 'end_time': datetime.datetime(2023, 5, 19, 15, 30)}

##### Select One Session for Tetrodotoxin (key5)


In [16]:
ephys_sessions_15min_query & "drug_name='Tetrodotoxin'"

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00
O09,2023-05-19 15:45:00,0,2023-05-20 15:20:00,2023-05-20 15:35:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00


In [17]:
key5 = (
    ephys_sessions_15min_query
    & "drug_name='Tetrodotoxin'"
    & "start_time='2023-05-19 15:50:00'"
).fetch1("KEY")
key5

{'organoid_id': 'O09',
 'experiment_start_time': datetime.datetime(2023, 5, 19, 15, 45),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 5, 19, 15, 50),
 'end_time': datetime.datetime(2023, 5, 19, 16, 5)}

##### Final Selection


In [18]:
organoid09_keys = [key1, key2, key3, key4, key5]
ephys_sessions_15min_query & organoid09_keys

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00
O09,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O09,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00


##### **1.2: Example Keys for the Remaining Organoids Under Different Conditions**

In [19]:
# Query one ephys session of 15min per condition for all organoids
organoid_id_list = np.unique(culture.Experiment.fetch("organoid_id"))
organoid_id_list[0]

organoids_keys = []
for organoid_id in organoid_id_list:
    organoid_query = (
        culture.Experiment * ephys.EphysSession
        & f"organoid_id='{organoid_id}'"
        & "session_type='spike_sorting'"
    ).proj(
        experiment_end_time="experiment_end_time",
        drug_name="drug_name",
        duration="TIMEDIFF(end_time, start_time)",
    ) & 'duration LIKE "%15%"'

    organoid_keys = organoid_query.fetch("KEY")
    indices = [1, 2, 4, 6, 7]
    organoid_keys = [organoid_keys[i] for i in indices]
    organoids_keys.append(organoid_keys)

organoids_query = (culture.Experiment * ephys.EphysSession & organoids_keys).proj(
    experiment_end_time="experiment_end_time",
    drug_name="drug_name",
    duration="TIMEDIFF(end_time, start_time)",
) & 'duration LIKE "%15%"'
organoids_query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00
O09,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O09,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00
O10,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O10,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O10,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00
O10,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O10,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00


##### **1.3: Queries for Batches Under Different Conditions**

In [20]:
batch1 = "'O09', 'O10', 'O11', 'O12'"
batch2 = "'O13', 'O14', 'O15', 'O16'"
batch3 = "'O17', 'O18', 'O19', 'O20'"

In [21]:
# query for batch 1
batch1_query = organoids_query & f"organoid_id IN ({batch1})"
batch1_query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O09,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O09,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O09,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00
O09,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O09,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00
O10,2023-05-03 17:33:00,0,2023-05-18 11:55:00,2023-05-18 12:10:00,2023-05-18 12:15:00,Control,0:15:00
O10,2023-05-18 12:25:00,0,2023-05-18 12:30:00,2023-05-18 12:45:00,2023-05-18 18:15:00,4-AP,0:15:00
O10,2023-05-18 18:15:00,0,2023-05-19 09:10:00,2023-05-19 09:25:00,2023-05-19 09:30:00,No Drug,0:15:00
O10,2023-05-19 09:30:00,0,2023-05-19 15:15:00,2023-05-19 15:30:00,2023-05-19 15:35:00,Bicuculline,0:15:00
O10,2023-05-19 15:45:00,0,2023-05-19 15:50:00,2023-05-19 16:05:00,2023-05-20 15:40:00,Tetrodotoxin,0:15:00


In [22]:
# query for batch 2
batch2_query = organoids_query & f"organoid_id IN ({batch2})"
batch2_query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O13,2023-05-25 19:18:00,0,2023-06-08 18:35:00,2023-06-08 18:50:00,2023-06-08 18:55:00,Control,0:15:00
O13,2023-06-08 19:05:00,0,2023-06-08 19:10:00,2023-06-08 19:25:00,2023-06-08 23:35:00,4-AP,0:15:00
O13,2023-06-08 23:35:00,0,2023-06-09 10:00:00,2023-06-09 10:15:00,2023-06-09 10:20:00,No Drug,0:15:00
O13,2023-06-09 10:20:00,0,2023-06-09 15:15:00,2023-06-09 15:30:00,2023-06-09 15:35:00,Bicuculline,0:15:00
O13,2023-06-09 15:40:00,0,2023-06-09 15:45:00,2023-06-09 16:00:00,2023-06-10 14:15:00,Tetrodotoxin,0:15:00
O14,2023-05-25 19:18:00,0,2023-06-08 18:35:00,2023-06-08 18:50:00,2023-06-08 18:55:00,Control,0:15:00
O14,2023-06-08 19:05:00,0,2023-06-08 19:10:00,2023-06-08 19:25:00,2023-06-08 23:35:00,4-AP,0:15:00
O14,2023-06-08 23:35:00,0,2023-06-09 10:00:00,2023-06-09 10:15:00,2023-06-09 10:20:00,No Drug,0:15:00
O14,2023-06-09 10:20:00,0,2023-06-09 15:15:00,2023-06-09 15:30:00,2023-06-09 15:35:00,Bicuculline,0:15:00
O14,2023-06-09 15:40:00,0,2023-06-09 15:45:00,2023-06-09 16:00:00,2023-06-10 14:15:00,Tetrodotoxin,0:15:00


In [23]:
# query for batch 3
batch3_query = organoids_query & f"organoid_id IN ({batch3})"
batch3_query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,experiment_end_time,drug_name,duration calculated attribute
O17,2023-06-22 15:40:00,0,2023-07-12 12:40:00,2023-07-12 12:55:00,2023-07-12 13:00:00,Control,0:15:00
O17,2023-07-12 13:10:00,0,2023-07-12 13:15:00,2023-07-12 13:30:00,2023-07-12 16:10:00,4-AP,0:15:00
O17,2023-07-12 16:10:00,0,2023-07-13 12:15:00,2023-07-13 12:30:00,2023-07-13 12:35:00,No Drug,0:15:00
O17,2023-07-13 12:35:00,0,2023-07-13 17:50:00,2023-07-13 18:05:00,2023-07-13 18:10:00,Bicuculline,0:15:00
O17,2023-07-13 18:15:00,0,2023-07-13 18:20:00,2023-07-13 18:35:00,2023-07-14 15:45:00,Tetrodotoxin,0:15:00
O18,2023-06-22 15:40:00,0,2023-07-12 12:40:00,2023-07-12 12:55:00,2023-07-12 13:00:00,Control,0:15:00
O18,2023-07-12 13:10:00,0,2023-07-12 13:15:00,2023-07-12 13:30:00,2023-07-12 16:10:00,4-AP,0:15:00
O18,2023-07-12 16:10:00,0,2023-07-13 12:15:00,2023-07-13 12:30:00,2023-07-13 12:35:00,No Drug,0:15:00
O18,2023-07-13 12:35:00,0,2023-07-13 17:50:00,2023-07-13 18:05:00,2023-07-13 18:10:00,Bicuculline,0:15:00
O18,2023-07-13 18:15:00,0,2023-07-13 18:20:00,2023-07-13 18:35:00,2023-07-14 15:45:00,Tetrodotoxin,0:15:00


#### **Step 2: Fetching the corresponding Intan Files for an Ephys Session**

In [24]:
key1 = {
    "organoid_id": "O15",
    "experiment_start_time": datetime.datetime(2023, 5, 25, 19, 18),
    "insertion_number": 0,
    "start_time": datetime.datetime(2023, 6, 8, 18, 35),
    "end_time": datetime.datetime(2023, 6, 8, 18, 50),
}
key1

{'organoid_id': 'O15',
 'experiment_start_time': datetime.datetime(2023, 5, 25, 19, 18),
 'insertion_number': 0,
 'start_time': datetime.datetime(2023, 6, 8, 18, 35),
 'end_time': datetime.datetime(2023, 6, 8, 18, 50)}

In [25]:
query = (
    culture.Experiment().proj("drug_name") * ephys.EphysSession
    & {"session_type": "spike_sorting"}
    & key1
)
query

organoid_id e.g. O17,experiment_start_time,insertion_number,start_time,end_time,drug_name,session_type
O15,2023-05-25 19:18:00,0,2023-06-08 18:35:00,2023-06-08 18:50:00,Control,spike_sorting


In [26]:
# These are the corresponding intan files for this session.

key = query.fetch1()
title = "_".join(
    [
        key["organoid_id"],
        key["start_time"].strftime("%Y%m%d%H%M"),
        key["end_time"].strftime("%Y%m%d%H%M"),
        key["drug_name"].replace(" ", ""),
    ]
)
files, file_times = (
    ephys.EphysRawFile
    & f"file_time BETWEEN '{key['start_time']}' AND '{key['end_time']}'"
).fetch("file_path", "file_time", order_by="file_time")

[print(file) for file in files]
print(f"\nNumber of files: {len(files)} ({key['drug_name']})")

O13-16_raw/processed_O13-16_230608_183542.rhd
O13-16_raw/processed_O13-16_230608_183642.rhd
O13-16_raw/processed_O13-16_230608_183742.rhd
O13-16_raw/processed_O13-16_230608_183842.rhd
O13-16_raw/processed_O13-16_230608_183942.rhd
O13-16_raw/processed_O13-16_230608_184042.rhd
O13-16_raw/processed_O13-16_230608_184142.rhd
O13-16_raw/processed_O13-16_230608_184242.rhd
O13-16_raw/processed_O13-16_230608_184342.rhd
O13-16_raw/processed_O13-16_230608_184442.rhd
O13-16_raw/processed_O13-16_230608_184542.rhd
O13-16_raw/processed_O13-16_230608_184642.rhd
O13-16_raw/processed_O13-16_230608_184742.rhd
O13-16_raw/processed_O13-16_230608_184842.rhd
O13-16_raw/processed_O13-16_230608_184942.rhd

Number of files: 15 (Control)
